In [1]:
import pyspark
import os.path

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from getpass import getpass
import pandas as pd
import pickle

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
fpathA = 'ecommerce/order_items_dataset.csv'
fpathB = 'ecommerce/customers_dataset.csv'
fpathC = 'ecommerce/geolocation_dataset.csv'
fpathD = 'ecommerce/order_payments_dataset.csv'
fpathE = 'ecommerce/orders_dataset.csv'
fpathF = 'ecommerce/products_dataset.csv'
fpathG = 'ecommerce/sellers_dataset.csv'
fpathH = 'ecommerce/product_category_name_translation.csv'
fpathI = 'ecommerce/customer_reviews_dataset.csv'

orders = pd.read_csv(fpathE)
orderItems = pd.read_csv(fpathA)
customerReviews = pd.read_csv(fpathI)
products = pd.read_csv(fpathF)

In [3]:
from datetime import  datetime
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])
orders['order_customer_delivery_date'] = pd.to_datetime(orders['order_customer_delivery_date'])
duration = (orders.order_customer_delivery_date - orders.order_estimated_delivery_date).astype(int)
duration = pd.Series.to_frame(duration)
orders['duration'] = duration

In [4]:
mergedOrdersDF = orders[['order_id', 'order_status', 'duration']]
orderItemsDF = orderItems[['order_id', 'product_id', 'price', 'freight_value']]
customerReviewsDF = customerReviews[['order_id', 'survey_score']]
productDF = products[['product_id', 'product_category_name']]

In [5]:
tempDF = orderItemsDF.merge(productDF, on=['product_id'], how='outer')
tempDF2 =  mergedOrdersDF.merge(tempDF, on=['order_id'], how='outer')
mergedDF = customerReviewsDF.merge(tempDF2, on=['order_id'], how='outer')

In [6]:
mergedDF.count()

order_id                 114100
survey_score             114100
order_status             114100
duration                 114100
product_id               113322
price                    113322
freight_value            113322
product_category_name    111710
dtype: int64

In [7]:
mergedDF = mergedDF.dropna(how='any')
mergedDF = mergedDF[['survey_score', 'order_status', 'duration', 'product_id', 'price', 'freight_value', 'product_category_name']]
mergedDF.count()

survey_score             111710
order_status             111710
duration                 111710
product_id               111710
price                    111710
freight_value            111710
product_category_name    111710
dtype: int64

In [8]:
from keras import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras_pandas.Automater import Automater
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [9]:
# Train /test split
train_observations, test_observations = train_test_split(mergedDF, test_size=0.1, random_state=42)
train_observations = train_observations.copy()
test_observations = test_observations.copy()
train_observations.head()

,survey_score,order_status,duration,product_id,price,freight_value,product_category_name
97592,5,delivered,-448290000000000,6e4d0d0f95ee20729970abe6ad1bbe39,79.90,16.81,utilidades_domesticas
82644,5,delivered,-1061294000000000,8c19dcc6a7f4c4259c45514b974b6441,65.00,52.81,perfumaria
7501,4,delivered,-785871000000000,5bd6493a2d5a384987386f2696bae33d,18.98,15.10,telefonia
55378,5,delivered,-708602000000000,ec2d43cc59763ec91694573b31f1c29a,45.90,16.11,cama_mesa_banho
63200,2,delivered,-376267000000000,6e0b3bc6a8bcfcca9005f9d220fe4af8,76.80,11.74,moveis_decoracao
